In [ ]:
import sys
sys.path.append("..")

from pandas import DataFrame
import pandas as pd
import numpy as np
from jmetal.util.visualization import CDplot
import matplotlib.pyplot as plt
import random
import re
import os
import git
from scipy.stats import wilcoxon
from sqlalchemy import create_engine, or_, Column, Integer, String, Float, DateTime, ForeignKey, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from do_analysis import Experiment, ExperimentSet

In [ ]:
def from_db_to_pandas(query):
    last = ""
    headers = []
    datasets = []
    series = []
    serie = None
    code_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
    repo = git.Repo(code_directory)

    for experiment in query:
        if last != experiment.file_name:
            last = experiment.file_name
            dataset_name = experiment.file_name.rsplit('/')[-1].split(".")[0]
            datasets.append(dataset_name)
            if serie is not None:
                series.append(serie)
            serie = []
        message = experiment.set.description
        measure =  experiment.method.split('.')[-1]
        if "Learning Based" in message:
            continue
        
        if "Initial" not in experiment.set.description:
            header = measure.replace('Modified','')
        else:
            header = measure
            
            
        if header not in headers:
            headers.append(header)
        if experiment.number_of_classes is None or experiment.number_of_clusters != experiment.number_of_classes:
            serie.append(np.nan)
        else:
            serie.append(experiment.f_score)
    
    series.append(serie)
    print(len(headers))
    
    return pd.DataFrame(series, index = datasets, columns=headers)

In [ ]:
engine = create_engine('sqlite:///../results.db', echo=False)
session_class = sessionmaker(bind=engine)
session = session_class()
query = session.query(Experiment).order_by(Experiment.file_name, Experiment.id)
#query = for experiment in session.query(Experiment).filter(or_(Experiment.set_id==i for i in [17])).order_by(Experiment.file_name):
#query = for experiment in session.query(Experiment).filter(Experiment.number_of_clusters == Experiment.number_of_classes).order_by(Experiment.file_name):
df = from_db_to_pandas(query)
df

In [ ]:
# Discard when Kappa is low and multiplying for normal weight	
# Discard when Kappa is low and multiplying for 1 - weight	
# Maximum when Kappa is low and multiplying for normal weight
# Maximum when Kappa is low and multiplying for 1 - weight
for measure in ["Eskin", "Gambaryan", "Goodall", "OF", "IOF", "Lin"]:
    configs = ["-w K -o D -t N", "-w K -o D -t I", "-w K -o M -t N", "-w K -o M -t I"]
    
    pattern = re.compile(f"^{measure}\s")
    all_headers = [x for x in df.columns if pattern.match(x)]
    #all_headers = [x for x in df.columns if measure in x]
    headers = []
    for config in configs:
        for header in all_headers:
            if config in header:
                headers.append(header)
    #headers = headers +
    partial = df[headers]
    partial = partial.fillna(0.0)
    CDplot(partial.transpose(), higherIsBetter=True, alpha=0.05)

In [ ]:
for measure in ["Eskin"]:
    configs = ["-w K -o D -t N", "-w K -o D -t I", "-w K -o M -t N", "-w K -o M -t I"]
    
    pattern = re.compile(f"^{measure}\s")
    all_headers = [x for x in df.columns if pattern.match(x)]
    #all_headers = [x for x in df.columns if measure in x]
    headers = []
    for config in configs:
        for header in all_headers:
            if config in header:
                headers.append(header)
    #headers = headers +
    partial = df[headers]
partial